In [25]:
import pandas as pd

In [29]:
user_data     = pd.read_csv('user_data.csv', low_memory=False, parse_dates=['purchase_date_utc'])
subscriptions = pd.read_csv('subscriptions.csv', low_memory=False, parse_dates=['order_created_at'])

In [31]:
user_data['customer_account_id']    = pd.to_numeric(user_data['customer_account_id'], errors='coerce').astype('Int64')
subscriptions['customer_account_id']= pd.to_numeric(subscriptions['customer_account_id'], errors='coerce').astype('Int64')

In [33]:
subs_success = subscriptions[subscriptions['invoice_status'] == 'success']

In [35]:
cohort = 
user_data[(user_data['purchase_date_utc'].dt.year == 2024) &(user_data['purchase_date_utc'].dt.month == 7)].copy()
cohort_ids = cohort['customer_account_id'].dropna().astype(int).unique()

In [37]:
subs_cohort = 
(subs_success[subs_success['customer_account_id'].isin(cohort_ids)].merge
(cohort[['customer_account_id','purchase_date_utc','age','geo_group','device_type']],on='customer_account_id', how='left'))

In [39]:
subs_cohort['days_since'] = (subs_cohort['order_created_at'] - subs_cohort['purchase_date_utc']).dt.days

In [41]:
ltv_per_user = (subs_cohort[subs_cohort['days_since'] <= 365].groupby('customer_account_id')['order_amount'].sum().reset_index(name='ltv_12m'))

In [43]:
ltv_per_user = ltv_per_user.merge(cohort[['customer_account_id','age','geo_group','device_type']],on='customer_account_id', how='left')

In [45]:
CAC_JULY = 60.93

In [47]:
ltv_by_age = ( ltv_per_user.groupby('age')['ltv_12m'].mean().reset_index())
ltv_by_age['ltv_cac_ratio'] = ltv_by_age['ltv_12m'] / CAC_JULY

In [49]:
segment_ltv = (ltv_per_user.groupby(['geo_group','device_type'])['ltv_12m'].mean().reset_index())
best_segment = segment_ltv.loc[segment_ltv['ltv_12m'].idxmax()]

In [53]:
print("1) LTV:CAC ratio by age segment:")
print(ltv_by_age.to_string(index=False), "\n")

print("2) Top-performing segment (geo_group & device_type):")
print(f"   geo_group: {best_segment['geo_group']}, "
      f"device_type: {best_segment['device_type']}, "
      f"avg 12m LTV: ${best_segment['ltv_12m']:.2f}\n")

print("3) Recommendation:")
print("   Focus acquisition & product efforts on the above segment and the 45+ age group, "
      "which show the highest LTV:CAC and absolute LTV.")

1) LTV:CAC ratio by age segment:
  age    ltv_12m  ltv_cac_ratio
18-24  64.918426       1.065459
25-34  88.812752       1.457619
35-44 115.464363       1.895033
  45+ 142.632001       2.340916 

2) Top-performing segment (geo_group & device_type):
   geo_group: NON_ELITE_T1, device_type: other, avg 12m LTV: $131.28

3) Recommendation:
   Focus acquisition & product efforts on the above segment and the 45+ age group, which show the highest LTV:CAC and absolute LTV.
